In [1]:
import numpy as np
import pandas as pd

### Loading Data

In [14]:
df = pd.read_csv('data/raw/games.csv')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71716 entries, 0 to 71715
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       71716 non-null  int64  
 1   Name                        71715 non-null  object 
 2   Release date                71716 non-null  object 
 3   Estimated owners            71716 non-null  object 
 4   Peak CCU                    71716 non-null  int64  
 5   Required age                71716 non-null  int64  
 6   Price                       71716 non-null  float64
 7   DLC count                   71716 non-null  int64  
 8   About the game              69280 non-null  object 
 9   Supported languages         71716 non-null  object 
 10  Full audio languages        71716 non-null  object 
 11  Reviews                     9167 non-null   object 
 12  Header image                71716 non-null  object 
 13  Website                     350

# Data Cleaning

In [18]:
drop_columns = [
    'Header image', 'Website', 'Support url', 'Support email',
    'Metacritic url', 'Screenshots', 'Movies', 'Score rank',
    'Notes', 'Estimated owners', 'Peak CCU', 'DLC count',
    'Full audio languages', 'Reviews',
    'Average playtime two weeks', 'Median playtime two weeks',
    'Support email', 'Support url'
]
df_clean = df.drop(columns=drop_columns)

In [20]:
df_clean.columns.tolist()

['AppID',
 'Name',
 'Release date',
 'Required age',
 'Price',
 'About the game',
 'Supported languages',
 'Windows',
 'Mac',
 'Linux',
 'Metacritic score',
 'User score',
 'Positive',
 'Negative',
 'Achievements',
 'Recommendations',
 'Average playtime forever',
 'Median playtime forever',
 'Developers',
 'Publishers',
 'Categories',
 'Genres',
 'Tags']